Using Lottery.txt dataset,

Please find top most commonly drawn triple numbers.

(9,38,42) -> 2 times

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

In [97]:
from pyspark import SparkContext

# import file
sc = SparkContext.getOrCreate()
lotteryRDD = sc.textFile("Lottery.txt")

# def split_data(text):
#   info = text.replace("}{", "},{")
#   return (info)

# lotteryRDD = lotteryRDD.map(lambda line: split_data(line))

lotteryRDD = lotteryRDD.flatMap(lambda x: x.split(','))
lotteryRDD = lotteryRDD.filter(lambda line: "rakamlarNumaraSirasi" in line)

lotteryRDD.take(5)

[' "rakamlarNumaraSirasi": "5 - 15 - 20 - 32 - 34 - 47"',
 ' "rakamlarNumaraSirasi": "3 - 14 - 16 - 23 - 27 - 43"',
 ' "rakamlarNumaraSirasi": "6 - 22 - 26 - 31 - 38 - 44"',
 ' "rakamlarNumaraSirasi": "7 - 11 - 16 - 18 - 25 - 26"',
 ' "rakamlarNumaraSirasi": "2 - 5 - 14 - 30 - 31 - 39"']

In [98]:
def format_lines(text):
  info = text.replace("\"rakamlarNumaraSirasi\": \"", "").split(" - ")
  info[0]= info[0].replace(" ", "")
  info[5]= info[5].replace("\"", "")
  return (info)

lotteryRDD = lotteryRDD.map(lambda line : format_lines(line))
lotteryRDD.take(10)

[['5', '15', '20', '32', '34', '47'],
 ['3', '14', '16', '23', '27', '43'],
 ['6', '22', '26', '31', '38', '44'],
 ['7', '11', '16', '18', '25', '26'],
 ['2', '5', '14', '30', '31', '39'],
 ['2', '14', '24', '25', '30', '36'],
 ['4', '8', '14', '24', '27', '36'],
 ['3', '17', '36', '41', '46', '47'],
 ['4', '15', '16', '18', '27', '41'],
 ['5', '11', '24', '29', '41', '45']]

In [99]:
from itertools import combinations
# Use lambda function to create groups of 3
lotteryRDD = lotteryRDD.flatMap(lambda x: (list(combinations(x, 3))))
lotteryRDD = lotteryRDD.map(lambda x:(x,1))
lotteryRDD = lotteryRDD.reduceByKey(lambda x,y:x+y)
sortRDD = lotteryRDD.sortBy(lambda x: x[1], ascending=False)
lotteryRDD.take(10)

[(('5', '15', '32'), 3),
 (('5', '15', '47'), 3),
 (('5', '20', '34'), 2),
 (('5', '32', '47'), 1),
 (('15', '20', '34'), 1),
 (('15', '32', '47'), 2),
 (('20', '32', '34'), 1),
 (('20', '34', '47'), 3),
 (('3', '14', '16'), 3),
 (('3', '23', '27'), 2)]

In [100]:
most_winners= sortRDD.take(1)
most_winners

[(('12', '19', '47'), 8)]

In [101]:
print(" Top most commonly drawn triple numbers in lottery winners: " ,most_winners[0][0], " for ", most_winners[0][1]," times" )

 Top most commonly drawn triple numbers in lottery winners:  ('12', '19', '47')  for  8  times
